In [1]:
import pandas as pd
import numpy

In [3]:
vsetko = pd.read_csv("data/opravene_spojene.csv")

In [4]:
vsetko.head()

,Unnamed: 0,name,address,FTI measured,education,lithium,TT4,T4U,capital-loss,capital-gain,...,query hyperthyroid,TT4 measured,on antithyroid medication,FTI,sick,T4U measured,psych,query hypothyroid,age,sex
0,0,Aaron Johansen,"3642 Melissa Crest\nNew Alicia, KS 60976",True,HS-grad,False,67.0,0.93,2001.0,0.0,...,False,True,False,72.0,False,True,False,False,1.3,M
1,1,Abigail Martinez,"5009 Lynch Highway\nSharonmouth, KS 23172",True,HS-grad,False,103.0,1.05,0.0,0.0,...,False,True,False,98.0,False,True,False,False,34.0,F
2,2,Abraham Bruce,"49824 Kim View\nWest Ericborough, VT 42457",True,Some_college,False,91.0,0.80,0.0,3908.0,...,False,True,False,113.0,False,True,False,False,73.0,M
3,3,Abraham Hicks,Unit 3701 Box 5448\nDPO AP 04047,True,Prof-school,False,71.0,1.00,0.0,0.0,...,False,True,False,71.0,False,True,False,False,55.0,M
4,4,Ada Jeffries,Unit 8721 Box 6441\nDPO AP 00654,True,Some_college,False,112.0,0.99,0.0,0.0,...,False,True,False,113.0,False,True,False,False,80.0,F
